In [1]:
import pandas as pd
import json
import re
import requests

from bs4 import BeautifulSoup
from googlesearch import search
from datetime import date

ModuleNotFoundError: No module named 'googlesearch'

In [ ]:
def query_google():
    
    query = "{} money control"
    my_results_list = []
    for j in list_yet_remaining:
        for i in search(query.format(j),        # The query you want to run
                        tld = 'com',  # The top level domain
                        lang = 'en',  # The language
                        num = 1,     # Number of results per page
                        start = 0,    # First result to retrieve
                        stop = 1,  # Last result to retrieve
                        pause = 2.0,  # Lapse between HTTP requests
                       ):
            my_results_list.append(i)
    
    return my_results_list

In [ ]:
# Get the nifty list and url
nifty_200_list =pd.read_csv('nifty_list/nifty200list.csv')['Symbol']
nifty_200_list_url = query_google()

# The mapping for index and url and convert to string
index_url_mapping = (dict(zip(list_yet_remaining, my_results_list)))
dumped_index_url_mapping = json.dumps(index_url_mapping)

# Save the mapping to a json
with open('index_url_mapping.json', 'w') as outfile:
    json.dump(dumped_index_url_mapping, outfile)

In [2]:
# Fetch the mapping from the Jsonn and evaluate
index_url_mapping_json_file = open('index_url_mapping.json')
index_url_mapping_data = json.load(index_url_mapping_json_file)
index_url_mapping = eval(index_url_mapping_data)

In [3]:
def get_and_parse_url(url):
        try:
            result = requests.get(url)
            soup = BeautifulSoup(result.text, 'html.parser')
            stk_name = soup.find("h1", attrs={'class': 'pcstname'})
            stock_name = stk_name.text
            all_data = soup.find("div", attrs={'class': 'nsert'})
            m = all_data.text.split("\n")
            comm_sentiment = soup.find("div", attrs={'class': 'commounity_senti'})
            community_sentiment = comm_sentiment.text
            tech_rating = soup.find("div", attrs={'class': 'mt15 CTR pb20'})
            technical_rating = tech_rating.find('a')['title']
            valuation = soup.find_all("div", attrs={'class': 'value_txtfr'})
            p_e = valuation[1].text
            list_bsh = re.findall("parseInt\('(.*?)'\);", community_sentiment)
            
            buy_percentage = list_bsh[0]  if list_bsh else -1
            sell_percentage = list_bsh[1] if list_bsh else -1
            hold_percentage = list_bsh[2] if list_bsh else -1
            
            community_sentiment_data = [buy_percentage, sell_percentage, hold_percentage]
            while "" in m:
                m.remove("")
            data_of_one_url = make_dataframe(technical_rating, m, stock_name, community_sentiment_data, p_e)
            return data_of_one_url
        except Exception as e:
            print(url)
            print(e)

In [4]:
def make_dataframe(technical_rating, data, stock_name, community_sentiment_data, p_e):
        # Calling DataFrame constructor
        df = pd.DataFrame()
        try:
            df = df.append({"Stock_Name": stock_name,
                            "Price_Change": data[3],
                            "Prev_close": data[14],
                            "Open_Price": data[16],
                            "52_Week_Low": data[20],
                            "52_Week_High": data[22],
                            "Technical_Rating": technical_rating,
                            "buy_percentage": community_sentiment_data[0],
                            "sell_percentage": community_sentiment_data[1],
                            "hold_percentage": community_sentiment_data[2],
                            "p_e": p_e
                            }, ignore_index=True)
        except:
            print ("error found {}".format(df))
        return df

In [5]:
def get_current_stock_market_data():
    current_stock_data_df = pd.DataFrame()

    for i in index_url_mapping.values():
        temp_stock_df = get_and_parse_url(i)

        if current_stock_data_df.empty:
            current_stock_data_df = temp_stock_df
        else:
            current_stock_data_df = pd.concat([current_stock_data_df, temp_stock_df], axis=0, ignore_index=True)
    
    return current_stock_data_df
        
        
    

In [6]:
current_stock_data_df = get_current_stock_market_data()

https://www.moneycontrol.com/india/fnoquote/tatamtrdvr/TMD/2019-08-29
'NoneType' object has no attribute 'text'


In [7]:
current_stock_data_df

,52_Week_High,52_Week_Low,Open_Price,Prev_close,Price_Change,Stock_Name,Technical_Rating,buy_percentage,hold_percentage,p_e,sell_percentage
0,1769.05,895.15,1172.00,1187.60,39.50 (3.33%),ACC Ltd.,Very Bullish,100,0,17.19,0
1,1218.00,380.15,415.00,420.90,-21.00 (-4.99%),AU Small Finance Bank Ltd.,Very Bearish,50,0,18.09,50
2,430.60,203.00,315.00,317.15,-6.55 (-2.07%),Adani Ports and Special Economic Zone Ltd.,Very Bullish,0,0,32.63,100
3,73.80,23.00,33.50,34.25,-1.25 (-3.65%),Adani Power Ltd.,Neutral,67,0,-,33
4,366.40,147.50,178.40,178.80,2.05 (1.15%),Adani Transmission Ltd.,Very Bearish,-1,-1,3617,-1
5,115.35,37.35,42.75,43.20,-1.20 (-2.78%),Aditya Birla Capital Ltd.,Very Bearish,100,0,-,0
6,285.50,97.00,111.50,112.10,-0.95 (-0.85%),Aditya Birla Fashion & Retail Ltd.,Bearish,100,0,43.59,0
7,1580.00,825.00,1491.00,1487.40,24.85 (1.67%),Ajanta Pharma Ltd.,Very Bullish,-1,-1,29.92,-1
8,2879.00,1660.00,2475.00,2474.95,22.95 (0.93%),Alkem Laboratories Ltd.,Neutral,-1,-1,25.22,-1
9,813.85,348.55,600.00,596.40,-3.85 (-0.65%),Amara Raja Batteries Ltd.,Very Bullish,100,0,15.73,0
